In [1]:
from modules.base import *

#### Graph state 다루기

- build 한 그래프에 접근해서 다룰수 있는 몇가지 메서드 

    - get_graph : 그래프 노드, 엣지 정보 확인

    - get_state : 그래프 상태 정보확인 (config 필수)

        - 입력 : `RunnableConfig` 필수

        - 리턴값 : ***StateSnapshot***

    - get_state_history : 그래프 상태 정보 히스토리 확인 (config 필수)

        - 입력 : `RunnableConfig` 필수

        - 리턴값 : ***StateSnapshot*** 제너레이터 (반복문으로 접근 필요)
        
    - update_state : 그래프 상태정보 업데이트
        
        - 입력 : `RunnableConfig` 필수

        - 리턴값 : `RunnableConfig`  

- 참고

    - https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.state.CompiledStateGraph.aget_state
    
    - https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/time-travel/#interacting-with-the-agent

- 알아둘 내용 ✅

    - 멀티턴을 위해 ShortTermMemory를 사용할때, config 정보를 넣어주게 됨.
    
    - graph 실행시 해당 config 정보에 대해 `checkpoint_ns, checkpoint_id` 가 생성됨

    - 그래프에서 노드가 순차적으로 실행될때, 주고 받는 상태정보(state)에 대한 인식/구별은 `checkpoint_id` 로 수행함.

        - 참고로 `state['messages']`에 있는 id값과 별개이다.

        - 즉 요약하면

            - `messages`에 있는 `id` 값은 `메시지간 인식을 위한것`

            - `config`에 있는 `checkpoint_id` 는 그래프의 `노드간 state 구별을 위한것`

    - 아래 예시 참고 : 3번 대화한후 마지막 ***상태정보(snapshot)*** 를 추출한 것

```json
{
  "StateSnapshot": {
    "values": {
      "messages": [{"HumanMessage": {"content": "안녕 나는 창우라고해", "id": "72a56ee6-8054-462e-a419-30a688306418" ...중략...}},
                   {"AIMessage": {"content": "안녕하세요, 창우님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?", "id": "run-b71776ac-76e4-478e-adfd-cd70b5271f5a-0" ...중략...}},
                   {"HumanMessage": {"content": "나는 30살이고 인공지능을 공부하고있어.", "id": "c9c86e76-c1ec-4d6d-b1e4-0686fc4cfcee" ...중략...}},
                   {"AIMessage": {"content": "멋지네요! 인공지능은 정말 흥미로운 분야죠. 어떤 부분을 공부하고 계신가요? ...", "id": "run-bfd772b8-6b73-42ff-aeaf-14fae2b64f10-0" ...중략...}},
                   {"HumanMessage": {"content": "나에 대해 아는게있어?", "id": "11ca4ea2-2d73-4c9d-a469-fa7fe463ab7e" ...중략...}},
                   {"AIMessage": {"content": "당신에 대해 알고 있는 정보는 대화 중에 공유해 주신 내용뿐입니다. 예를 들어, 당신의 이름이 창우이고, 30살이며...", "id": "run-ce108968-6155-4316-9e04-1daeab7cb276-0" ...중략...}}]},
    "next": [],
    "config": {
      "configurable": {
        "thread_id": "initial_chat",
        "checkpoint_ns": "",
        "checkpoint_id": "1efcf246-7089-6e04-8007-da621da0c2cb"
      }
    },
    "parent_config": {
      "configurable": {
        "thread_id": "initial_chat",
        "checkpoint_ns": "",
        "checkpoint_id": "1efcf246-5ede-638b-8006-e76f1cfe3378"
      }
    },
    "metadata": {.. 중략 ..}
    "created_at": "2025-01-10T07:27:52.616487+00:00",
  }
}
```

---

##### 1. 기본 테스트

In [2]:
@trace_function()
def node_answer(state:MessagesState)->MessagesState:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(MessagesState)
builder.add_node("node_answer", node_answer)
builder.add_edge(START, "node_answer")
builder.add_edge("node_answer", END)
ShortTermMemory = MemorySaver()
graph = builder.compile(checkpointer=ShortTermMemory)

In [3]:
config = {"configurable": {"thread_id": "initial_chat", 
                           "user_id": "changwoo"}}

In [4]:
graph.invoke({"messages":"안녕 나는 창우라고해"}, config)
graph.invoke({"messages":"나는 30살이고 인공지능을 공부하고있어."}, config)
graph.invoke({"messages":"나에 대해 아는게있어?"}, config)


🚀 Passing Through [node_answer] ..

🚀 Passing Through [node_answer] ..

🚀 Passing Through [node_answer] ..


{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='1ca55b54-c6e2-409a-9448-6016a1b4fc4c'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-3712dacf-f581-4592-abbb-22fcba0befa2-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 30살이고 인공지능을 공부하고있어.', additional_kwargs={}, response_metadata={}, id='65d3918f-

In [5]:
graph.get_graph(config)

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'node_answer': Node(id='node_answer', name='node_answer', data=node_answer(tags=None, recurse=True, func_accepts_config=False, func_accepts={'writer': False, 'store': False}), metadata=None), '__end__': Node(id='__end__', name='__end__', data=<class 'langchain_core.utils.pydantic.LangGraphOutput'>, metadata=None)}, edges=[Edge(source='__start__', target='node_answer', data=None, conditional=False), Edge(source='node_answer', target='__end__', data=None, conditional=False)])

In [6]:
snapshot = graph.get_state(config)
snapshot.values['messages']

[HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='1ca55b54-c6e2-409a-9448-6016a1b4fc4c'),
 AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-3712dacf-f581-4592-abbb-22fcba0befa2-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='나는 30살이고 인공지능을 공부하고있어.', additional_kwargs={}, response_metadata={}, id='65d3918f-4a62-4a3d-b814-

In [7]:
state_history = graph.get_state_history(config)
for snapshot in state_history:
    print(f"{YELLOW}snapshot(상태,state) 마다 config의 checkpoint가 다릅니다. 현재 snapshot의 config : {snapshot.config}{RESET}\n")
    for message in snapshot.values['messages']:
        print(message.content)
    print(f"{RED}----------------------------------{RESET}")

snapshot(상태,state) 마다 config의 checkpoint가 다릅니다. 현재 snapshot의 config : {'configurable': {'thread_id': 'initial_chat', 'checkpoint_ns': '', 'checkpoint_id': '1efd16ac-7281-6a23-8007-7fc6510cd7f4'}}

안녕 나는 창우라고해
안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?
나는 30살이고 인공지능을 공부하고있어.
멋지네요! 인공지능은 다양한 분야에서 많은 가능성을 열어주는 흥미로운 주제죠. 특정 분야나 주제에 대해 공부 중이신가요? 아니면 전반적인 개념을 배우고 계신가요? 도움이 필요하시면 언제든지 말씀해 주세요!
나에 대해 아는게있어?
죄송하지만, 저는 개인 정보에 접근할 수 없습니다. 창우님께서 공유해 주신 정보만을 바탕으로 대화할 수 있습니다. 혹시 더 알고 싶거나 공유하고 싶은 것이 있으면 말씀해 주세요!
----------------------------------
snapshot(상태,state) 마다 config의 checkpoint가 다릅니다. 현재 snapshot의 config : {'configurable': {'thread_id': 'initial_chat', 'checkpoint_ns': '', 'checkpoint_id': '1efd16ac-687b-6309-8006-ce357aaa5f34'}}

안녕 나는 창우라고해
안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?
나는 30살이고 인공지능을 공부하고있어.
멋지네요! 인공지능은 다양한 분야에서 많은 가능성을 열어주는 흥미로운 주제죠. 특정 분야나 주제에 대해 공부 중이신가요? 아니면 전반적인 개념을 배우고 계신가요? 도움이 필요하시면 언제든지 말씀해 주세요!
나에 대해 아는게있어?
----------------------------------
snapshot(상태,state) 마다 config의 checkpoint

##### 2. replay 테스트

- 대화 기록중 특정 snapshot 부분만 가져와서, 그래프를 재실행함.

    - `입력값은 동일`하다.

- 중요한 점은 `재실행하려는 부분의 config`를 넣어줘야한다.

- 이전 기록은 삭제됨. 

In [8]:
state_history = [s for s in graph.get_state_history(config)]
snapshot_to_replay_from_initial = state_history[-2] # 제일 첫 입력값이 나옴.
for message in snapshot_to_replay_from_initial.values['messages']:
    print(message.content)

안녕 나는 창우라고해


In [9]:
graph.invoke(None, snapshot_to_replay_from_initial.config)


🚀 Passing Through [node_answer] ..


{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='1ca55b54-c6e2-409a-9448-6016a1b4fc4c'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-af4291d7-e5cb-454f-b9db-626a25ef09d2-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [10]:
snapshot = graph.get_state(config)
for message in snapshot.values['messages']:
    print(message.content)

안녕 나는 창우라고해
안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?


##### 3. forking 테스트

- forking은 `입력값을 바꾸어` 그래프를 재실행하는걸 의미함.

- 중요한 점은 `입력값을 바꾸고 재실행하려는 부분의 config`를 넣어줘야한다.

- 이전 기록은 삭제됨. 

In [11]:
state_history = [s for s in graph.get_state_history(config)]
snapshot_to_replay_with_new_value = state_history[-2] # 제일 첫 입력값이 나옴.
for message in snapshot_to_replay_with_new_value.values['messages']:
    print(message.content)

안녕 나는 창우라고해


In [12]:
fork_config = graph.update_state(snapshot_to_replay_with_new_value.config, 
                                 {"messages": [HumanMessage(content="안녕 나는 홍길동이라고해.",
                                                            id=snapshot_to_replay_with_new_value.values['messages'][0].id)]})

In [13]:
graph.invoke(None, fork_config)


🚀 Passing Through [node_answer] ..


{'messages': [HumanMessage(content='안녕 나는 홍길동이라고해.', additional_kwargs={}, response_metadata={}, id='1ca55b54-c6e2-409a-9448-6016a1b4fc4c'),
  AIMessage(content='안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 16, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-d510d4c5-6183-47b2-bf47-965ea46952eb-0', usage_metadata={'input_tokens': 16, 'output_tokens': 22, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}